# Recurrent Neural Networks for time series
RNN is a neural network that contains recurrent layers, design to consequently process sequence of inputs.
With RNN we can feed in a batch of sequences and it will output a batch of forecasts.

The difference from previous NN is that input shape is 3-dimensional: batch size, time steps, dimensionality of inputs at each time step.
At each time step `i` memory cell takes the input value for that step and state input, and then calculates the output of that step (`Yi`) and state factor (`Hi`), which is then feed into the next memory cell.
Values recur due to the output of the cell at one step being fed back into itself at the next state.

![Recurrent Layer](./recurrent_layer.png)

## Shape of the inputs to the RNN
The inputs are 3-dimensional. For example if we have a window size of 30 timesteps and we are batching them in sizes of 4, the shape would be 4x30x1.
At each timestep the memory cell input (`Xi`) will be a 4x1 matrix. The memory cell will output the `Hi` value aka state matrix.
Cell will also output a `Yi` value. If the memory cell is comprised of 3 neurons, then the output matrix will be 4x3 because batch size was 4 and a number of neurons is 3.
So the whole output of the neuron is 3-dimensional, in this case: 4x30x3. With 4 being the batch size, 3 being a number of units and 30 being a number of overall steps.

In a simple RNN the state output of H is just the copy of output matrix Y (e.g. H0 is a copy of Y0, H1 is a copy of Y1 adn so on).
At each step the memory cell gets both the current input Xi and also the previous output Yi.

### Sequence-to-Vector
In some cases you want to input a sequence, but you do not want a sequence as an on output. This is called a `Sequence-to-Vector` RNN.
In reality all you do is just ignore all previous output values of Y except the last one (e.g. Y29). In TF this is a default behaviour, otherwise you have to specify `return_sequences=true` to make layer return a sequence.

![Sequence to vector](./sequence-to-vector.png)

## Outputting a sequence

In [ ]:
import tensorflow as tf
model = tf.keras.models.Sequential([
    # Note input_shape dimension below:
    # None - batch size, can be any size so no need to define it
    # None - number of time steps, which means RNN can handle sequences of any length
    # 1 - a single dimension for our univariate time series data
    tf.keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    tf.keras.layers.SimpleRNN(20), # no sequences will be output, only the output state will be returned
    tf.keras.layers.Dense(1)
])

If we set `return_sequences` to true in all recurrent layers, it will give us a `sequence-to-sequence` RNN: is fed a batch of sequences and it returns a batch of sequences of the same length.
In this case Dense layer will get a sequence as an input as well. TF handles it by using the same Dense layer independently at each timestep.
The dimensionality may not always match, it depends on a number of units in a memory cell.


## Lambda layers
Lambda layers allow us to perform arbitrary operations to effectively expand the functionality of TF Keras. We can do it within the model definition itself.

The **first** lambda layer will be used to help us with dimensionality.
In a simple NN there were 2 dimensions of data: batch size and number of time steps. In RNN there are 3 dimensions on input data: batch size, number of time steps and series dimensionality. with lambda we can fix this without rewriting our `window_dataset` helper function: we just expand array with 1 dimension using lambda.

The **second** lambda layer will help us with scaling output values. This can help training, because a default activation function in RNN layers is `tanh` which outputs values between -1 and 1. Since time series values are in this order (usually 10s, 40s, 60s etc.) then scaling up the outputs can help us with learning. So we simply multiply the output by hundred.

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=-1), input_shape=[None]),
    tf.keras.layers.SimpleRNN(20, return_sequences=True),
    tf.keras.layers.SimpleRNN(20),
    tf.keras.layers.Dense(1),
    tf.keras.layers.Lambda(lambda x: x * 100.0)
])


## Adjusting the learning rate dynamically
An example of RNN with two recurrent layers each of 40 cells. To tune a learning rate we set up a callback using `LearningRateScheduler`.
We use a [Huber loss](https://en.wikipedia.org/wiki/Huber_loss) function which is less sensitive to outliers, handy when data is noisy.

In [ ]:
train_set = windowed_dataset(x_train, window_size, batch_size=128, shuffle_buffer=shuffle_buffer_size)

model = tf.keras.models.Sequential([
    tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=-1), input_shape=[None]),
    tf.keras.layers.SimpleRNN(40, return_sequences=True),
    tf.keras.layers.SimpleRNN(40),
    tf.keras.layers.Dense(1),
    tf.keras.layers.Lambda(lambda x: x * 100.0)
])

lr_schedule = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 1e-8 * 10**(epoch / 20))
optimizer = tf.keras.optimizers.SGD(lr=1e-8, momentum=0.9)

model.compile(loss=tf.keras.losses.Huber(), optimizer=optimizer, metrics=["mae"])

history = model.fit(train_set, epochs=100, callbacks=[lr_schedule])

## LSTM (Long short-tern memory)
In RNNs the state factor impact can diminish quickly over time steps.
LSTMs add a cell state to this that keep a state throughout the life of the training so that the state is passed from cell to cell, time step to time step and can be better maintained.
That means that the data earlier in a window can have a greater impact on an overall projection than in the case of RNNs.
The state can also be a bi-directional, so that it moves forwards and backwards. In the case of texts it is very powerful, but in case of predicting numerical sequences it might not be.

![LSTM](./LSTM.png)

## Coding LSTMs

In [ ]:
tf.keras.backend.clear_session() # clear any internal variables, makes easy to experiment
dataset = windowed_dataset(x_train, window_size, batch_size=128, shuffle_buffer=shuffle_buffer_size)

model = tf.keras.models.Sequential([
    tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, axis= -1), input_shape=[None]),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32), return_sequences=True), # A single bi-directional LSTM layers with 32 cells
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32), return_sequences=True), # A single bi-directional LSTM layers with 32 cells
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)), # A single bi-directional LSTM layers with 32 cells
    tf.keras.layers.Dense(1), # gives prediction value
    tf.keras.layers.Lambda(lambda x: x * 100.0)
])

model.compile(loss="mse", optimizer=tf.keras.optimizers.SGD(lr=1e-6, momentum=0.9))
model.fit(dataset, epochs=100, verbose=0)

